Load dependencies


In [1]:
import tensorflow as tf
import math
import numpy as np
import time

In [2]:
SEED = 2022
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [3]:
def generate_synthetic_data():
    [a, b, c] = np.random.rand(3)
    a = a if a != 0 else np.finfo(float).eps
    radicant = b**2 - (4 * a * c)
    x_1_real = x_2_real = -b / (2 * a)
    if radicant >= 0:
        sqrt_term = math.sqrt(radicant) / (2 * a)
        x_1_real += sqrt_term
        x_2_real -= -b / (2 * a)
        x_1_im = x_2_im = 0
    else:
        x_1_im = math.sqrt(abs(radicant)) / (2 * a)
        x_2_im = -x_1_im
    return a, b, c, x_1_real, x_1_im, x_2_real, x_2_im


In [4]:
NB_SAMPLES = 1000000
NB_POLYNOMIAL_COEFFS = 3  # a, b, c
NB_SOLUTION_COEFFS = 4  # (real, imaginary) twice + is_real
features = np.empty((NB_SAMPLES, NB_POLYNOMIAL_COEFFS))
labels = np.empty((NB_SAMPLES, NB_SOLUTION_COEFFS))
for i in range(NB_SAMPLES):
    a, b, c, x_1_real, x_1_im, x_2_real, x_2_im = generate_synthetic_data()
    features[i] = [a, b, c]
    labels[i] = [x_1_real, x_1_im, x_2_real, x_2_im]


In [5]:
def custom_loss_function(y_true, y_pred):
    solution_1_dist = tf.reduce_sum(tf.square(y_true[0:2:] - y_pred[0:2:]))
    solution_2_dist = tf.reduce_sum(tf.square(y_true[2:4:] - y_pred[2:4:]))
    return tf.reduce_mean([solution_1_dist, solution_2_dist], axis=-1)


In [6]:
Sequential = tf.keras.Sequential
Dropout = tf.keras.layers.Dropout
Dense = tf.keras.layers.Dense
DROPOUT = 0.5


model = Sequential()
model.add(Dense(8, input_shape=(3,), activation="gelu"))
model.add(Dense(16, activation="gelu"))
model.add(Dense(32, activation="gelu"))
model.add(Dropout(DROPOUT))
model.add(Dense(16, activation="gelu"))
model.add(Dropout(DROPOUT))
model.add(Dense(8, activation="gelu"))
model.add(Dense(4))
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=custom_loss_function,
    metrics=[custom_loss_function],
)

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 32        
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 32)                544       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_4 (Dense)             (None, 8)                 1

In [7]:
BATCH_SIZE = 1000
EPOCHS = 1000
VALIDATION_SPLIT = 0.1
PATIENCE = 100

callback = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=PATIENCE)

model.fit(
    x=features,
    y=labels,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    shuffle=True,
    epochs=EPOCHS,
    callbacks=[callback],
)

model.save("./model/model.h5")

Epoch 1/1000
900/900 [==============================] - 6s 5ms/step - loss: 3.0147 - custom_loss_function: 3.0147 - val_loss: 1.3103 - val_custom_loss_function: 1.3103
Epoch 2/1000
900/900 [==============================] - 4s 4ms/step - loss: 2.0646 - custom_loss_function: 2.0646 - val_loss: 0.9643 - val_custom_loss_function: 0.9643
Epoch 3/1000
900/900 [==============================] - 4s 5ms/step - loss: 1.5973 - custom_loss_function: 1.5973 - val_loss: 0.8114 - val_custom_loss_function: 0.8114
Epoch 4/1000
900/900 [==============================] - 4s 4ms/step - loss: 1.4836 - custom_loss_function: 1.4836 - val_loss: 0.7962 - val_custom_loss_function: 0.7962
Epoch 5/1000
900/900 [==============================] - 4s 4ms/step - loss: 1.3394 - custom_loss_function: 1.3394 - val_loss: 0.6213 - val_custom_loss_function: 0.6213
Epoch 6/1000
900/900 [==============================] - 4s 4ms/step - loss: 1.0820 - custom_loss_function: 1.0820 - val_loss: 0.5780 - val_custom_loss_function:

In [8]:
model.evaluate(features, labels)

31250/31250 [==============================] - 51s 2ms/step - loss: 0.3750 - custom_loss_function: 0.3750


[0.37503480911254883, 0.37503480911254883]

In [9]:
MODEL_DIR = "./model/model.h5"
custom_objects = {"custom_loss_function": custom_loss_function}
a = 0.5
b = 0.5
c = 0.5
model = tf.keras.models.load_model(MODEL_DIR, custom_objects=custom_objects)
tic = time.perf_counter()
out_prediction = model.predict(tf.expand_dims([a, b, c], axis=0))
toc = time.perf_counter()
x_1_real, x_1_im, x_2_real, x_2_im = out_prediction[0]
print(f"{a}x^2+{b}x+{c}")
print(f"x_1={x_1_real}{x_1_im:+f}j\nx_2={x_2_real}{x_2_im:+f}j")
print(f"Inference time {toc - tic:0.4f} seconds")


0.5x^2+0.5x+0.5
x_1=-0.41996389627456665+0.893589j
x_2=-0.48808664083480835-0.893588j
Inference time 0.1651 seconds
